In [6]:
import os
import requests
from typing import Any, Type

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.tools import DuckDuckGoSearchResults

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-0125")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        # Web Search API
        # brew install ddgr
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )

# 기업 개요
class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}" # 기업 개요 api
        )
        return r.json()
    
# 손익
class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]

# 주가
class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


"""
agent에 input을 줄 때 runtime은 LLM으로 부터 get_action을 호출해서 LLM이 선택한 Tool을 돌림
AgentFinish가 될떄까지 loop를 돌리기 떄문에 Cost가 많이 나올 수 있음.
가끔 LLM이 Json이 아닌 Text로 답할 때가 있음. 그러땐 Agent가 에러가 남
"""
# llm, tool, agent를 전달해야 함.
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS, # OpenAI Agent
    handle_parsing_errors=True, # LLM의 output 때문에 Parsing Error가 발생하면 이를 수정하려고 함.
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        # CompanyStockPerformanceTool(), # Token이 많음..
    ],
)


prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment. Tell me Korean"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare'}`


[snippet: Read the latest posts from Cloudflare experts and leaders on topics such as email security, hardware automation, zero trust, AI, and more. Learn how Cloudflare protects and enhances the Internet with its products and services., title: The Cloudflare Blog, link: https://blog.cloudflare.com/], [snippet: Cloudflare's Year in Review revealed that in 2023 alone, there was a 25% increase in global traffic, with no signs of slowing. This growth underscores the dependency on Internet services to facilitate and underpin vital systems and tasks such as supporting global digital economies, enabling the operations of healthcare networks, maintaining ..., title: Cloudflare Publishes Top Internet Trends for 2023, link: https://www.cloudflare.com/press-releases/2023/press-release/cloudflare-publishes-top-internet-trends-for-2023/], [snippet: Finally, because Cloudflare also oper

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment. Tell me Korean",
 'output': '클라우드플레어(Cloudflare)는 미국의 기술 기업으로, 전 세계 기업들에게 다양한 네트워크 서비스를 제공하는 클라우드 플랫폼을 운영합니다. 회사는 샌프란시스코에 본사를 두고 있으며, NYSE에서 "NET"으로 거래되고 있습니다.\n\n여러 가지 재무 정보를 살펴보면, 클라우드플레어의 최근 매출은 12억 9674만 5000달러이며, 순 손익은 -1억 8394만 9000달러입니다. 영업 이익은 -1억 8548만 5000달러이며, 총 자산은 3조 218억 9034만달러입니다. 주당 순 손익은 -0.55달러이며, 주당 순 자산은 2.261달러입니다.\n\n주식 시장에서의 클라우드플레어의 주가는 52주 최고가가 116달러이고, 52주 최저가가 39.9달러입니다. 현재 50일 이동평균은 94.69달러이며, 200일 이동평균은 74.93달러입니다.\n\n클라우드플레어의 주식 투자 가능성을 분석하기 위해서는 더 많은 정보와 시장 동향을 고려해야 합니다. 투자 결정을 내리기 전에 전문가의 조언을 참고하고, 시장 조사를 신중히 진행하는 것이 중요합니다.'}